# nova-selachiia — Inventário e blueprint

Este notebook transforma o inventário do projeto em um **blueprint executável**: escopo, hipóteses, contrafactuais, métricas e um esqueleto mínimo para começar a implementação sem overengineering.

## 1) Motivação
- Científica: entender como mudanças em drivers ambientais/ antrópicos alteram dinâmica, persistência e risco de colapso de Selachiia na costa brasileira.
- Pessoal: recorte local (Brasil), dados melhores, relevância aplicada.

## 2) Pergunta central
Se variáveis ambientais e/ou antrópicas mudarem em Δ% (intervenção contrafactual), como isso altera a dinâmica temporal, probabilidade de sobrevivência e padrões de colapso de espécies-chave de tubarões no Atlântico Sudoeste?

Diagnóstico: o sistema é determinístico, estocástico ou híbrido?

## 3) Hipóteses (implícitas)
- **H1**: dinâmica populacional depende de variáveis ambientais (ex.: SST).
- **H2**: pressão antrópica altera média e regime (↑ estocasticidade).
- **H3**: respostas a perturbações ±Δ% são assimétricas (não-lineares).
- **H4**: sistema híbrido: componente determinístico médio + ruído ecológico relevante.

## 4) Restrições (reais)
- Notebook pessoal (CPU/RAM limitadas; GPU opcional).
- Projeto enxuto (semanas a poucos meses).
- Costa brasileira; 3–6 espécies; 5–10 presas/guildas; grid ~1°×1°; séries mensais.
- Evitar overengineering (sem transformers/GNNs grandes/HPC).

## 5) Formulação do problema (inputs/outputs)
**Entradas candidatas**
- Ambiental: SST (principal), anomalia ΔSST, tendência local (opcionais adicionais: clorofila-a/O₂ se não pesar).
- Presas: proxies de abundância relativa via OBIS (agregado por célula/mês; normalizar por esforço quando possível).
- Antrópico: fishing_pressure (GFW ideal; FAO como alternativa coarse).
- Ruído: variabilidade intrínseca tratada como feature.

**Saídas esperadas**
- Séries por espécie × célula: índice de ocorrência normalizado (proxy de abundância relativa).
- Métricas: $P_{surv}(δX,T)$, tempo até colapso, entropia/variância do ensemble, sensitividade $∂Y/∂X$.

## 6) Contrafactual (definição e plausibilidade)
Para intervenção $δX$ aplicada a um driver $X$:

- Gerar trajetórias $Y'(t)$ condicionadas ao mesmo estado inicial $z_{t0}$ e ao driver modificado:

$$X'(t) = X(t)⋅(1+δ).$$

**Plausível**: dentro de limites históricos/físicos/ecológicos, preserva invariantes (não-negatividade, taxa máxima de variação).
**Inválido**: extrapola além de limites observacionais ou viola invariantes.

## 7) Arquitetura (tese de trabalho)
- **NSSM (determinístico)**: captura dinâmica média interpretável.
- **DMM condicional (estocástico)**: adiciona σz/σy e gera ensembles MC contrafactuais.

Interpretação: **ruído é feature**; comparar cenários A/B via métricas definidas.

In [9]:
# Config mínima do projeto (editável)
from nova_selachiia.config import ProjectConfig, RegionBBox, as_tuple

cfg = ProjectConfig(
    region=RegionBBox(lat_min=-35.0, lat_max=5.0, lon_min=-50.0, lon_max=-30.0),
    grid_deg=1.0,
    shark_species=as_tuple([
        "Prionace glauca",           # tubarão-azul (pelágico, muito capturado)
        "Carcharhinus longimanus",   # galha-branca-oceânico (CR, CITES)
        "Sphyrna lewini",            # martelo-recortado (CR)
        "Isurus oxyrinchus",         # mako (EN, alta pressão de pesca)
        "Carcharhinus leucas",       # cabeça-chata (costeiro, estuarino)
        "Galeocerdo cuvier",         # tigre (topo de cadeia)
    ]),
    prey_groups=as_tuple([
        "Clupeidae",      # sardinhas (forrageio pelágico)
        "Engraulidae",    # anchovas
        "Scombridae",     # atuns/cavalas (pelágicos)
        "Carangidae",     # xaréus
        "Loliginidae",    # lulas costeiras
        "Octopodidae",    # polvos (bentônicos)
        "Trichiuridae",   # peixe-espada
        "Mugilidae",      # tainhas (costeiro/estuarino)
    ]),
    collapse_threshold_quantile=0.10,
    mc_trajectories=200,
    random_seed=42,
    delta_scenarios=(-0.2, -0.1, 0.0, 0.1, 0.2),
)

cfg

ProjectConfig(region=RegionBBox(lat_min=-35.0, lat_max=5.0, lon_min=-50.0, lon_max=-30.0), grid_deg=1.0, monthly=True, shark_species=('Prionace glauca', 'Carcharhinus longimanus', 'Sphyrna lewini', 'Isurus oxyrinchus', 'Carcharhinus leucas', 'Galeocerdo cuvier'), prey_groups=('Clupeidae', 'Engraulidae', 'Scombridae', 'Carangidae', 'Loliginidae', 'Octopodidae', 'Trichiuridae', 'Mugilidae'), collapse_threshold_quantile=0.1, mc_trajectories=200, random_seed=42, delta_scenarios=(-0.2, -0.1, 0.0, 0.1, 0.2))

In [10]:
# Utilidades mínimas de contrafactual + métricas (baseline)
import numpy as np
from nova_selachiia.counterfactual import apply_delta_percent
from nova_selachiia.metrics import survival_probability, collapse_time

def auc_delta(y_factual: np.ndarray, y_counterfactual: np.ndarray) -> float:
    """Difference in area under curve (counterfactual - factual)."""
    y_factual = np.asarray(y_factual)
    y_counterfactual = np.asarray(y_counterfactual)
    # np.trapezoid é o novo nome no NumPy 2.0+ (antes era np.trapz)
    return float(np.trapezoid(y_counterfactual) - np.trapezoid(y_factual))


# Exemplo toy: y factual e contrafactual (substituir pelos outputs do modelo)
t = np.arange(120)
y = np.exp(-t / 80)  # decaimento lento
x_driver = 1.0 + 0.1 * np.sin(t / 12)

delta = 0.1
x_cf = apply_delta_percent(x_driver, delta)
y_cf = y * (1 + 0.2 * (x_cf - x_driver))  # placeholder

threshold = np.quantile(y, cfg.collapse_threshold_quantile)

print('threshold:', float(threshold))
print('collapse_time factual:', collapse_time(y, threshold))
print('collapse_time cf:', collapse_time(y_cf, threshold))
print('auc_delta:', auc_delta(y, y_cf))
print('survival_probability (single traj):', survival_probability(np.vstack([y_cf]), threshold))

threshold: 0.2621750175962157
collapse_time factual: 108.0
collapse_time cf: 109.0
auc_delta: 1.267022834887527
survival_probability (single traj): 0.0


## 8) Próximos passos (implementação mínima)
1. Implementar aquisição/agragação de dados: SST (NOAA) + OBIS (ocorrências) + fishing_pressure (FAO/GFW).
2. Definir proxy de abundância e tratamento de esforço.
3. Baseline NSSM simples por celula.
4. DMM condicional para gerar ensembles MC contrafactuais.
5. Métricas e diagnósticos de regime (variância/entropia, sensitividades).